### Dask Bag

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install --upgrade "dask[complete]"

In [10]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

/home/mayur/anaconda3/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40997 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40997/status,
Dashboard: http://127.0.0.1:40997/status,Workers: 4
Total threads: 4,Total memory: 31.37 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36851,Workers: 4
Dashboard: http://127.0.0.1:40997/status,Total threads: 4
Started: Just now,Total memory: 31.37 GiB
Comm: tcp://127.0.0.1:42089,Total threads: 1
Dashboard: http://127.0.0.1:33211/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:33539,


In [11]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                # Make records of people

In [12]:
b

dask.bag<mimesis, npartitions=10>

In [13]:
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/0.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/1.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/2.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/3.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/4.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/5.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/6.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/7.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/8.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/9.json']

In [14]:
!head -n 2 data/0.json

{"age": 65, "name": ["Daryl", "Wolfe"], "occupation": "Blacksmith", "telephone": "+16070572911", "address": {"address": "1015 Colon Boulevard", "city": "Oceanside"}, "credit-card": {"number": "2611 0866 1854 2225", "expiration-date": "03/23"}}
{"age": 25, "name": ["Jerrica", "O'neill"], "occupation": "Letting Agent", "telephone": "+13601874433", "address": {"address": "876 Powhattan Circle", "city": "Spartanburg"}, "credit-card": {"number": "4683 3915 1376 2756", "expiration-date": "04/21"}}


In [15]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [16]:
b.take(2)

({'age': 65,
  'name': ['Daryl', 'Wolfe'],
  'occupation': 'Blacksmith',
  'telephone': '+16070572911',
  'address': {'address': '1015 Colon Boulevard', 'city': 'Oceanside'},
  'credit-card': {'number': '2611 0866 1854 2225',
   'expiration-date': '03/23'}},
 {'age': 25,
  'name': ['Jerrica', "O'neill"],
  'occupation': 'Letting Agent',
  'telephone': '+13601874433',
  'address': {'address': '876 Powhattan Circle', 'city': 'Spartanburg'},
  'credit-card': {'number': '4683 3915 1376 2756',
   'expiration-date': '04/21'}})

In [17]:
b.filter(lambda record: record['age'] > 50).take(2)

({'age': 65,
  'name': ['Daryl', 'Wolfe'],
  'occupation': 'Blacksmith',
  'telephone': '+16070572911',
  'address': {'address': '1015 Colon Boulevard', 'city': 'Oceanside'},
  'credit-card': {'number': '2611 0866 1854 2225',
   'expiration-date': '03/23'}},
 {'age': 51,
  'name': ['Darin', 'Erickson'],
  'occupation': 'Building Control',
  'telephone': '+1-412-617-9531',
  'address': {'address': '320 Severn Heights', 'city': 'Sachse'},
  'credit-card': {'number': '3480 241039 39089', 'expiration-date': '06/22'}})

In [18]:
b.map(lambda record: record['occupation']).take(2)

('Blacksmith', 'Letting Agent')

In [19]:
b.count().compute()  # Count total number of records

10000

In [20]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

In [21]:
result.compute()

[('Land Surveyor', 15),
 ('Mineralologist', 15),
 ('Blacksmith', 14),
 ('Adjustor', 14),
 ('Bodyshop', 14),
 ('Research Analyst', 14),
 ('Sales Engineer', 14),
 ('Operations', 13),
 ('Merchant Seaman', 13),
 ('Warehousewoman', 13)]

In [22]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.0.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.1.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.2.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.3.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.4.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.5.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.6.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.7.json',
 '/home/mayur/Documents/Explore Libraries/Explore-Libraries/ML-Computations-libs/dask/data/processed.8.json',
 '/home/ma